# Required vs Optional Fields

In [3]:
from pydantic import BaseModel

# To make a field optional look at center


class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int

We can also see this by inspecting the model fields:

In [2]:
Circle.model_fields

{'center': FieldInfo(annotation=tuple[int, int], required=False, default=(0, 0)),
 'radius': FieldInfo(annotation=int, required=True)}

We can now create instances of `Circle` without providing a value for `center`:

In [3]:
Circle(radius=1)

Circle(center=(0, 0), radius=1)

In [4]:
data = {"radius": 1}
data_json = '{"radius": 1}'

In [5]:
Circle.model_validate(data)

Circle(center=(0, 0), radius=1)

In [6]:
Circle.model_validate_json(data_json)

Circle(center=(0, 0), radius=1)

Of course, we can provide a value for center too:

In [7]:
Circle(center=(1, 1), radius=2)

Circle(center=(1, 1), radius=2)

We have to be quite careful about one thing when specifying a default value for a field.

When we provide a field value, Pydantic validates that value before putting it into our model instance. However, Pydantic's default behavior does not validate default values!

That kind of makes sense - after all, we are writing the model definition, so we should be able to only provide a valid default value in our model definition.

So, here's the issue:

In [4]:
class Model(BaseModel):
    field: int = "Python"

As you can see, the provided default is totally inconsistent for an `int` type, and yet this still works:

In [8]:
Model(field=10)

Model(field=10)

Usually in Python, including dataclasses, we have to be extra careful when we assign a default that is a mutable object.

let's look at an example of this with a regular Python function:

By the way, writing a function that behaves this way - modifies it's input and returns it, is a terrible coding technique - I'm just using this to illustrate a point.

We can call this function with our own list:

Huh??

This is the issue I was telling you about. When we defined a default for function arguments, these values are calculated and stored with the function itself - they are not re-created every time the function is called. In other words, that default value is going to be **shared** amogst all the function calls.

Something similar happens with dataclases. In dataclasses, we get around the problem by a mechanism that is called a default factory. Instead of providing a mutable object as a default value, we provide a function that will get called each time a dataclass instance is created, and that function will therefore return a **new** default object.

Pydantic is very similar, in that it offers this default factory idea (which we'll cover later).

However, it goes one step further, and actually allows us to simply define mutable objects as defaults. 

When Pydantic sees this, it will actually create a deep copy of the mutable object every time a new model instance is created.